In [74]:
# import libraries
import requests
import pandas as pd
import time
import math

In [75]:
# keys
CLIENT_ID = '******'
CLIENT_SECRET = '******'

# API call procedures
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [80]:
def convert_ms(duration_ms):  # change duration_ms to 'mm:ss' format
  m = math.floor((duration_ms/1000)/60)  
  s = math.floor((duration_ms/1000)%60)
  if s<10:
    s = '0'+str(s)
  duration = str(m)+':'+str(s)
  return duration


def get_track_details(track_id):

  # collect energy, loudness, tempo, and duration
  r_track = requests.get(BASE_URL+'audio-features/'+track_id, headers=headers)
  r_track = r_track.json()

  energy = r_track['energy']
  loudness = r_track['loudness']
  tempo = r_track['tempo']
  duration_ms = r_track['duration_ms']

  return energy, loudness, tempo, convert_ms(duration_ms)

In [77]:
def get_tracks(df, i):

  # first get playlist containing all of the tracks I listen to
  r = requests.get(BASE_URL+'users/dedwur/playlists', headers=headers)  # 'dedwur' is my spotify username
  r = r.json()
  playlist_id = r['items']['name'=='K']['id']  # I called this playlist 'K'

  # now get tracks from playlist
  r2 = requests.get(BASE_URL+'playlists/'+playlist_id+'/tracks?offset='+str(i*100), headers=headers)
  r2 = r2.json()

  for track in r2['items']:
    track_id = track['track']['id']
    track_title = track['track']['name']
    track_artists = track['track']['artists']
    artist_list = []
    for artist in track_artists:
      artist_list.append(artist['name'])
    track_artists = ", ".join(artist_list)
    track_album = track['track']['album']['name']
    track_release_date = track['track']['album']['release_date']
    energy, loudness, tempo, duration = get_track_details(track_id)
    df = df.append({'track_id':track_id,'track_title':track_title,
                    'track_artists':track_artists,'track_album':track_album,
                    'track_release_date':track_release_date,'energy':energy,
                    'loudness':loudness,'tempo':tempo,
                    'duration':duration}, ignore_index=True) 

  return df

In [81]:
# main
columns = ['track_id', 'track_title', 'track_artists', 'track_album', 'track_release_date', 'energy', 'loudness', 'tempo', 'duration']
df_all = []
for i in range(0,10):
  df = pd.DataFrame(columns=columns) 
  while True:
    try:
      df = get_tracks(df, i)
    except KeyError:
      continue
    break
  df_all.append(df)
df_all = pd.concat(df_all, ignore_index=True)

In [82]:
df_all

,track_id,track_title,track_artists,track_album,track_release_date,energy,loudness,tempo,duration
0,3RBXNeUbe3X7ZBch00D2St,Extraordinary Girl / Letterbomb,Green Day,American Idiot,2004-09-21,0.958,-4.051,137.921,7:40
1,2gTdDMpNxIRFSiu7HutMCg,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, Travis Barker",Hotel Diablo,2019-07-05,0.744,-4.718,119.921,2:49
2,6KnkYUz3zXQnftT1fzZcDL,다시 해줘 Do it again,TWICE,THE STORY BEGINS,2015-10-20,0.936,-0.948,136.983,3:26
3,0dy6iXYIF0piirySAzCBwF,The End Has No End,The Strokes,Room On Fire,2003-10-28,0.660,-3.725,136.027,3:05
4,7yCPwWs66K8Ba5lFuU2bcx,All The Small Things,blink-182,Enema Of The State,1999-01-01,0.891,-4.764,148.599,2:48
...,...,...,...,...,...,...,...,...,...
928,0ttHpMs250dpFkCYAnuhnE,Good,Better Than Ezra,Deluxe,1995-02-28,0.657,-6.873,109.950,3:05
929,13NCxLOlvQ4Tnexgfp03Gs,Do Me a Favour,Arctic Monkeys,Favourite Worst Nightmare (Standard Version),2007-04-24,0.735,-6.685,180.127,3:25
930,6tCssnvTUREcperDOUTqvA,WANNABE,ITZY,It'z Me,2020-03-09,0.911,-4.410,122.035,3:11
931,3yHQKddM8SVCRnuPSo3HPN,BOOMBAYAH,BLACKPINK,SQUARE ONE,2016-08-08,0.836,-3.282,124.969,4:00


In [84]:
df_all.to_csv('spotify_track_data.csv')